In [87]:
import pandas as pd
import numpy as np
import pickle

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.utils import np_utils
from keras.preprocessing.text import one_hot
from sklearn.preprocessing import LabelEncoder

In [88]:
train_df = pd.read_csv('D:/Program/dataset/Spooky_Author_Identification/train.csv')
test_df = pd.read_csv('D:/Program/dataset/Spooky_Author_Identification/test.csv')


# Label Onehot encoding  

le = LabelEncoder()
le.fit(train_df['author'])
print(le.classes_)

y = le.transform(train_df['author'])
y_train = np_utils.to_categorical(y)

y_train

['EAP' 'HPL' 'MWS']


array([[ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 1.,  0.,  0.],
       ..., 
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  1.,  0.]])

In [89]:
with open('keras_input_train.pkl', 'rb') as f:
    sequences = pickle.load(f)
with open('keras_input_test.pkl', 'rb') as f:
    test_sequences = pickle.load(f)
    
x_train = sequences
x_test = test_sequences
x_test.shape

(8392, 5000)

In [90]:
model = Sequential() 
model.add(Dense(units=200,input_shape=(5000,),activation="relu",name="hidden"))
model.add(Dense(units=200,activation="relu",name="hidden2"))
model.add(Dense(units=200,activation="relu",name="hidden3"))
model.add(Dense(units=3,activation="softmax",name="output"))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden (Dense)               (None, 200)               1000200   
_________________________________________________________________
hidden2 (Dense)              (None, 200)               40200     
_________________________________________________________________
hidden3 (Dense)              (None, 200)               40200     
_________________________________________________________________
output (Dense)               (None, 3)                 603       
Total params: 1,081,203
Trainable params: 1,081,203
Non-trainable params: 0
_________________________________________________________________


In [91]:
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=['accuracy'])

In [92]:
train_history = model.fit(x_train, y_train, validation_split=0.2, epochs=10, batch_size=20, verbose=1)

Train on 15663 samples, validate on 3916 samples
Epoch 1/10
15663/15663 [==============================] - 8s - loss: 0.5818 - acc: 0.7560 - val_loss: 0.4687 - val_acc: 0.8141
Epoch 2/10
15663/15663 [==============================] - 7s - loss: 0.2069 - acc: 0.9209 - val_loss: 0.5668 - val_acc: 0.8069
Epoch 3/10
15663/15663 [==============================] - 7s - loss: 0.0704 - acc: 0.9734 - val_loss: 0.8862 - val_acc: 0.7952
Epoch 4/10
15663/15663 [==============================] - 7s - loss: 0.0311 - acc: 0.9903 - val_loss: 1.1460 - val_acc: 0.7993
Epoch 5/10
15663/15663 [==============================] - 7s - loss: 0.0137 - acc: 0.9958 - val_loss: 1.4201 - val_acc: 0.7962
Epoch 6/10
15663/15663 [==============================] - 7s - loss: 0.0058 - acc: 0.9985 - val_loss: 1.6808 - val_acc: 0.7957
Epoch 7/10
15663/15663 [==============================] - 7s - loss: 0.0072 - acc: 0.9979 - val_loss: 1.7979 - val_acc: 0.7942
Epoch 8/10
15663/15663 [==============================] - 7s -

In [93]:
scores = model.evaluate(x_train, y_train)  
print()
print(scores)

19200/19579 [============================>.] - ETA: 0s
[0.35208999502240074, 0.95796516675726007]


In [94]:
prediction = model.predict(x_test)
print()
print("Show 10 prediction result :")  
print(prediction[:10])
print(prediction.shape)


Show 10 prediction result :
[[  3.89291666e-09   7.12325143e-10   1.00000000e+00]
 [  1.00000000e+00   7.02010586e-26   5.37893138e-33]
 [  8.26686674e-14   1.00000000e+00   4.89141876e-15]
 [  1.00000000e+00   1.59056157e-10   1.27124242e-10]
 [  9.99789774e-01   1.82043543e-04   2.82269993e-05]
 [  1.00000000e+00   6.89576118e-10   5.89197701e-12]
 [  5.27132191e-02   9.38391149e-01   8.89559835e-03]
 [  9.68595426e-09   1.26164679e-08   1.00000000e+00]
 [  1.00000000e+00   5.77363020e-20   2.59638711e-22]
 [  9.99973774e-01   6.15544664e-08   2.60612542e-05]]
(8392, 3)


In [95]:
# submission

submission = pd.DataFrame(prediction, columns=le.classes_)
submission["id"] = test_df["id"]
cols = submission.columns.tolist()
cols = cols[-1:]+cols[:-1]
submission = submission[cols]
submission.head(10)

,id,EAP,HPL,MWS
0,id02310,3.892917e-09,7.123251e-10,1.000000e+00
1,id24541,1.000000e+00,7.020106e-26,5.378931e-33
2,id00134,8.266867e-14,1.000000e+00,4.891419e-15
3,id27757,1.000000e+00,1.590562e-10,1.271242e-10
4,id04081,9.997898e-01,1.820435e-04,2.822700e-05
5,id27337,1.000000e+00,6.895761e-10,5.891977e-12
6,id24265,5.271322e-02,9.383911e-01,8.895598e-03
7,id25917,9.685954e-09,1.261647e-08,1.000000e+00
8,id04951,1.000000e+00,5.773630e-20,2.596387e-22
9,id14549,9.999738e-01,6.155447e-08,2.606125e-05


In [96]:
submission.to_csv('submission.csv',index=False)